In [1]:
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf

In [2]:
tf.__version__


'2.0.0'

In [3]:
f = h5py.File('SVHN_single_grey1.h5','r')
data = f.get('SVHN_single_grey1.h5')
data_as_array = np.array(data)
list(f.keys())



#data = pd.read_hdf('SVHN_single_grey1.h5')

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [4]:
x_test = np.array(f['X_test'])
x_test.dtype

dtype('<f4')

In [5]:
x_train = f['X_train']

In [6]:
y_test = np.array(f['y_test'])
y_train = f['y_train']

In [7]:
y_test.shape

(18000,)

In [8]:
x_train.shape

(42000, 32, 32)

In [9]:
y_test

array([1, 7, 2, ..., 7, 9, 2], dtype=uint8)

In [10]:
x_val = np.array(f['X_val'])
y_val = np.array(f['y_val'])

In [11]:
x_val.shape

(60000, 32, 32)

In [12]:
y_val.shape

(60000,)

### It is seen that the data gas been read correctly

In [13]:
trainY = tf.keras.utils.to_categorical(y_train, num_classes=10)
testY = tf.keras.utils.to_categorical(y_test, num_classes=10)
valY = tf.keras.utils.to_categorical(y_val, num_classes=10)

In [14]:
print(trainY.shape)
print('First 5 examples now are: ', trainY[0:2])

(42000, 10)
First 5 examples now are:  [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


### Target Data has been converted to catgorical variable  

## Deep Neural Network is built next. First a Vanilla DNN is built and then DNN with Batch Normalization is added. 
## Finally CNN is added to see if the accuracy is improved

### Vanilla Deep Neural Network

In [15]:
model = tf.keras.models.Sequential()

# Reshape data from 2D to 1D -> 28x28 to 784
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))

# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [16]:
# Hidden layers
model.add(tf.keras.layers.Dense(200, activation='relu', name='Layer_1'))
model.add(tf.keras.layers.Dense(100, activation='relu', name='Layer_2'))

# Dropout layer
#model.add(tf.keras.layers.Dropout(0.5))

# Hidden layers
model.add(tf.keras.layers.Dense(60, activation='relu', name='Layer_3'))
model.add(tf.keras.layers.Dense(30, activation='relu', name='Layer_4'))


In [17]:
#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax', name='Output'))

In [18]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
Layer_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
Layer_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
Layer_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
Layer_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
Output (Dense)               (None, 10)                3

In [20]:
model.fit(x_train, trainY, validation_data=(x_val, valY), epochs=30,
          batch_size = 32, shuffle='batch')

Train on 42000 samples, validate on 60000 samples
Epoch 1/30
42000/42000 [==============================] - 5s 128us/sample - loss: 1.8710 - accuracy: 0.3459 - val_loss: 1.3295 - val_accuracy: 0.5718
Epoch 2/30
42000/42000 [==============================] - 5s 122us/sample - loss: 1.1879 - accuracy: 0.6220 - val_loss: 0.9689 - val_accuracy: 0.6963
Epoch 3/30
42000/42000 [==============================] - 6s 134us/sample - loss: 0.9777 - accuracy: 0.6960 - val_loss: 0.9117 - val_accuracy: 0.7218
Epoch 4/30
42000/42000 [==============================] - 6s 136us/sample - loss: 0.8678 - accuracy: 0.7279 - val_loss: 0.7430 - val_accuracy: 0.7736
Epoch 5/30
42000/42000 [==============================] - 5s 126us/sample - loss: 0.7890 - accuracy: 0.7510 - val_loss: 0.6948 - val_accuracy: 0.7882
Epoch 6/30
42000/42000 [==============================] - 5s 131us/sample - loss: 0.7340 - accuracy: 0.7719 - val_loss: 0.6747 - val_accuracy: 0.7932
Epoch 7/30
42000/42000 [==========================

In [21]:
model.evaluate(x_test, testY, verbose = 2)

18000/1 - 1s - loss: 0.4966 - accuracy: 0.8327


[0.6806755538185437, 0.83272225]

### The training accuracy is 0.8985
### The validation accuracy is 0.8844
### The testing accuracy is 0.8327

# With Batch Normalization

In [22]:
model_batch = tf.keras.models.Sequential()

# Reshape data from 2D to 1D -> 28x28 to 784
model_batch.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))

# Normalize the data
model_batch.add(tf.keras.layers.BatchNormalization())

In [23]:
model_batch.add(tf.keras.layers.Dense(1000, activation='relu', name='Layer_1'))

# Normalize the data
model_batch.add(tf.keras.layers.BatchNormalization())
model_batch.add(tf.keras.layers.Dense(500, activation='relu', name='Layer_2'))

# Normalize the data
model_batch.add(tf.keras.layers.BatchNormalization())

# Dropout layer
#model.add(tf.keras.layers.Dropout(0.5))

# Hidden layers
model_batch.add(tf.keras.layers.Dense(250, activation='relu', name='Layer_3'))

# Normalize the data
model_batch.add(tf.keras.layers.BatchNormalization())
model_batch.add(tf.keras.layers.Dense(125, activation='relu', name='Layer_4'))


In [24]:
#Output layer
model_batch.add(tf.keras.layers.Dense(10, activation='softmax', name='Output'))

In [25]:
model_batch.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model_batch.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
Layer_1 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1000)              4000      
_________________________________________________________________
Layer_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 500)               2000      
_________________________________________________________________
Layer_3 (Dense)              (None, 250)              

In [27]:
model_batch.fit(x_train, trainY, validation_data=(x_val, valY), epochs=30,
          batch_size = 32, shuffle='batch')

Train on 42000 samples, validate on 60000 samples
Epoch 1/30
42000/42000 [==============================] - 17s 401us/sample - loss: 1.4213 - accuracy: 0.5256 - val_loss: 0.9175 - val_accuracy: 0.7142
Epoch 2/30
42000/42000 [==============================] - 17s 412us/sample - loss: 0.9483 - accuracy: 0.6933 - val_loss: 0.7732 - val_accuracy: 0.7578
Epoch 3/30
42000/42000 [==============================] - 17s 397us/sample - loss: 0.7849 - accuracy: 0.7499 - val_loss: 0.6706 - val_accuracy: 0.7896
Epoch 4/30
42000/42000 [==============================] - 18s 421us/sample - loss: 0.6693 - accuracy: 0.7871 - val_loss: 0.5834 - val_accuracy: 0.8197
Epoch 5/30
42000/42000 [==============================] - 16s 380us/sample - loss: 0.5811 - accuracy: 0.8162 - val_loss: 0.5856 - val_accuracy: 0.8158
Epoch 6/30
42000/42000 [==============================] - 16s 392us/sample - loss: 0.5082 - accuracy: 0.8377 - val_loss: 0.5580 - val_accuracy: 0.8278
Epoch 7/30
42000/42000 [====================

In [28]:
model_batch.evaluate(x_test, testY, verbose = 2)

18000/1 - 1s - loss: 0.4895 - accuracy: 0.8362


[0.8554650974505478, 0.83622223]

### The training accuracy is 1.0000
### The validation accuracy is 0.9215
### The testing accuracy is 0.8362
##### Performance is slightly better than Vanilla network but overfitting is definately present

# CNN

In [29]:
tf.keras.backend.clear_session()
#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Reshape data from 2D (28,28) to 3D (28, 28, 1)
model.add(tf.keras.layers.Reshape((32,32,1),input_shape=(32,32,)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

In [30]:
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))

#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#Add MaxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

In [31]:
#Flatten the output
model.add(tf.keras.layers.Flatten())

In [32]:
#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add another dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [33]:
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 1)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [37]:
#Train the model
model.fit(x_train,trainY,          
          validation_data=(x_val,valY),
          epochs=15,
          batch_size=32, shuffle='batch')

Train on 42000 samples, validate on 60000 samples
Epoch 1/15
42000/42000 [==============================] - 72s 2ms/sample - loss: 0.3537 - accuracy: 0.8933 - val_loss: 0.2951 - val_accuracy: 0.9198
Epoch 2/15
42000/42000 [==============================] - 73s 2ms/sample - loss: 0.3205 - accuracy: 0.9028 - val_loss: 0.2697 - val_accuracy: 0.9270
Epoch 3/15
42000/42000 [==============================] - 74s 2ms/sample - loss: 0.2867 - accuracy: 0.9142 - val_loss: 0.2439 - val_accuracy: 0.9344
Epoch 4/15
42000/42000 [==============================] - 70s 2ms/sample - loss: 0.2618 - accuracy: 0.9203 - val_loss: 0.2315 - val_accuracy: 0.9396
Epoch 5/15
42000/42000 [==============================] - 70s 2ms/sample - loss: 0.2331 - accuracy: 0.9288 - val_loss: 0.2162 - val_accuracy: 0.9440
Epoch 6/15
42000/42000 [==============================] - 70s 2ms/sample - loss: 0.2152 - accuracy: 0.9343 - val_loss: 0.2025 - val_accuracy: 0.9474
Epoch 7/15
42000/42000 [==============================] 

In [38]:
model.evaluate(x_test, testY, verbose = 2)

18000/1 - 4s - loss: 0.4228 - accuracy: 0.9035


[0.47568908135096233, 0.9035]

### The training accuracy is 0.9660rchi
### The validation accuracy is 0.9662
### The testing accuracy is 0.9035
##### Performace is much better with CNN


#### Potential to improve
- Architecture
- SGD is used here but Adam might improve model

# Question/Note to moderator
- This notebook works as expected locally on all machines (Windows, Mac and Linux tested)
- All the models in this notebook stop learning (0.1 accuracy) on Google Colab
- Why is this happening?